In [205]:
import pandas as pd
import numpy as np
import datetime
import math
import nltk

In [206]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/james/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [207]:
df = pd.read_json('./json/karineJean-Pierre.json')
df = df.assign(keyword="biden")
dropList = ['id', 'dislikes', 'location', 'subtitles', 'comments', 'commentsTurnedOff', 'details', 'duration']
df = df.drop(dropList, axis =1)
df = df.replace(np.nan, 0)
df['ratio'] =  round((df['likes'] / df['viewCount'] + 0.1) * 2.5, 2)
df['ratio'] = df['ratio'].apply(lambda x: 1 / (1 + math.exp(-x)))

In [208]:
# THESE ARE THE SHEETS TOTALS
dfLength = len(df.index)
ratioAverage = df['ratio'].sum()
views = round(df['viewCount'].sum() / dfLength, 2)
likes = round(df['likes'].sum() / dfLength, 2)
comments = round(df['commentsCount'].sum() / dfLength, 2)
print(f'TOTAL VIDEOS: {dfLength}, WEIGHTED AV views: {views}, likes: {likes}, comments: {comments}, ratio: {ratioAverage}')

TOTAL VIDEOS: 173, WEIGHTED AV views: 1032124.43, likes: 19516.07, comments: 3720.16, ratio: 99.35171693958321


In [209]:
df['titleScores'] = df['title'].apply(lambda text: sid.polarity_scores(text))
df['titleCompound']  = df['titleScores'].apply(lambda score_dict: score_dict['compound'])
df['titleCompScore'] = df['titleCompound'].apply(lambda c: 'pos' if c >=0 else 'neg')
titleIndex = pd.Index(df['titleCompScore'])
titleIndex.value_counts()

pos    126
neg     47
Name: titleCompScore, dtype: int64

In [210]:
df['textScores'] = df['text'].apply(lambda text: sid.polarity_scores(text))
df['textCompound']  = df['textScores'].apply(lambda score_dict: score_dict['compound'])
df['textCompScore'] = df['textCompound'].apply(lambda c: 'pos' if c >=0 else 'neg')
df['date'] = df['date'].astype(str)
df['date'] = df['date'].str.slice(0,4)
textIndex = pd.Index(df['textCompScore'])
textIndex.value_counts()

pos    148
neg     25
Name: textCompScore, dtype: int64

In [211]:
df['sentimentScore'] = df['titleCompound'] + df['textCompound']
df['penalty'] =  (2023 - df['date'].astype(int)) *.05
df['score'] = ((df['commentsCount'].apply(lambda x: 1 / (1 + math.exp(-x))) + df['likes'].apply(lambda x: 1 / (1 + math.exp(-x)))  + df['viewCount'].apply(lambda x: 1 / (1 + math.exp(-x))))/100000) + 0.1 /3

In [212]:
df['score'] = (df['score'] * (df['ratio']) +0.5 * (df['sentimentScore'] +.3))
df['score'] = df['score'].apply(lambda x: 1 / (1 + math.exp(-x)))
df['score'] - df['penalty']
df = df.sort_values('score', ascending=False)
secondDrop = ['titleScores', 'titleCompScore', 'textScores', 'textCompScore', 'textCompound', 'titleCompound', 'ratio', 'penalty', 'sentimentScore', 'commentsCount', 'numberOfSubscribers', 'likes', 'date', 'viewCount' ]
df=df.drop(secondDrop, axis=1)

In [213]:
df.head(2)

,title,url,thumbnailUrl,channelName,channelUrl,text,keyword,score
98,President Obama Surprises Joe Biden With Medal...,https://www.youtube.com/watch?v=SzBFR2EE8hM&pp...,https://i.ytimg.com/vi/SzBFR2EE8hM/maxresdefau...,Wall Street Journal,http://www.youtube.com/@wsj,"In a special sendoff, President Barack Obama s...",biden,0.744419
121,'Play your hearts out:' Joe Biden wishes US sq...,https://www.youtube.com/watch?v=1Ks_yQUDqfw&pp...,https://i.ytimg.com/vi/1Ks_yQUDqfw/maxresdefau...,The Telegraph,http://www.youtube.com/@telegraph,US President Joe Biden called the US men's nat...,biden,0.729495


In [214]:
df.tail(2)

,title,url,thumbnailUrl,channelName,channelUrl,text,keyword,score
68,Russia-Ukraine War Live : President Joe Biden ...,https://www.youtube.com/watch?v=mE8tdOLRiXw&pp...,https://i.ytimg.com/vi/mE8tdOLRiXw/maxresdefau...,TV9 Bharatvarsh,http://www.youtube.com/@TV9Bharatvarsh,Russia-Ukraine War Live : President Joe Biden ...,biden,0.356868
43,Russia-Ukraine War : Putin का प्रतिशोध LIVE | ...,https://www.youtube.com/watch?v=ZlMTdsFITMs&pp...,https://i.ytimg.com/vi/ZlMTdsFITMs/maxresdefau...,TV9 Bharatvarsh,http://www.youtube.com/@TV9Bharatvarsh,Russia-Ukraine War : Putin का प्रतिशोध LIVE | ...,biden,0.349399


In [215]:
df.to_csv('karineJean-Pierre.csv', index=False)  

In [216]:
ls

biden.csv  data/  json/  notebooks/  README.md  tucker.ipynb
